In [ ]:
import pandas as pd

In [ ]:
data=pd.read_excel('data anticorps.xlsx')

In [ ]:
df=pd.read_excel('data_vaccin.xlsx')

In [ ]:
# Remplacer toutes les instances de "5J_7j" par 7 dans la colonne "jour_vaccinatio"
for i in range(len(df)):
    # Si la valeur dans la colonne 'jour_vaccinatio' est '5J_7j'
    if df.loc[i, 'jour_vaccin'] == ' 5j_7j':
        # Modifier la valeur de cette cellule à 7
        df.loc[i, 'jour_vaccin'] = 7

In [ ]:
df['jour'] = df['jour'].astype(int)
df['jour_vaccin'] = df['jour_vaccin'].astype(int)

In [ ]:
df['MDA '] = df.apply(lambda x: 0 if x['jour'] <  x['jour_vaccin']+14 else x['MDA '], axis=1)

In [ ]:
# Grouper les données par lot
grouped = df.groupby('lot ')

# Initialiser la liste qui contiendra les lots dont les MDA sont croissantes
lots_croissants = []

# Boucler sur chaque groupe
for name, group in grouped:
    # Obtenir le jour de vaccination
    jour_vaccin = group['jour_vaccin'].iloc[0]

    # Obtenir les données après jour_vaccin+14
    after_vaccin = group[group['jour'] > jour_vaccin + 14]

    # Vérifier si les MDA sont croissantes
    if after_vaccin['MDA '].is_monotonic_increasing:
        # Si oui, ajouter le groupe entier à la liste des lots croissants
        lots_croissants.append(group)

# Concaténer tous les groupes croissants en une seule DataFrame
data_reponse = pd.concat(lots_croissants, ignore_index=True)

In [ ]:
moyennes_par_jour = data_reponse.groupby('jour')['MDA '].mean()
liste_moyennes = moyennes_par_jour.tolist()
liste_moyennes

[0.0, 0.0, 0.0, 495.5, 752.0, 960.75]

In [ ]:
#ajustement degradation
import numpy as np
from scipy.optimize import curve_fit
# Définir la fonction exponentielle
def exp_func(x, a, b, c):
    return a * np.exp(-b * x) + c

# Entrer les données d'observation
x_data = data['jour']
y_data = data['MDA FLUNPS'] # Niveau d'anticorps observé

# Utiliser la méthode des moindres carrés pour ajuster la courbe
popt, pcov = curve_fit(exp_func, x_data, y_data)

# Afficher les paramètres a, b et c de la fonction ajustée
print('a1 =', popt[0], ', k1 =', popt[1], ', c1 =', popt[2])


a1 = 13625.854121665714 , k1 = 0.28738947824622907 , c1 = 595.9708514827894


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
#ajustement repone immunitaire
def log_func(x,a,k,c):

   return  a* np.log( x) +c

# Entrer les données d'observation
x_data =np.arange(0,22,7)+0.01   # Temps en semaines
y_data = np.array(liste_moyennes[2:] )# Niveau d'anticorps observé

# Utiliser la méthode des moindres carrés pour ajuster la courbe
#popt, pcov = curve_fit(exp_func, x_data, y_data)

pop , pcov = curve_fit(log_func, x_data  , y_data,bounds=(0,np.inf) )
pop  = np.asarray(pop , dtype=np.float64)

x_data = np.asarray(x_data, dtype=np.float64)
y_data = np.asarray(y_data, dtype=np.float64)
# Afficher les paramètres a, b et c de la fonction ajustée
print('A_0 =', pop [0], ', k =', pop [1], ', c =', pop [2])


A_0 = 107.94604781215682 , k = 1.0000000000000204 , c = 470.3723607087618


In [ ]:
A_0 = popt [0]
k = popt [1]
c = popt [2]
A_0 = popt [0]
k = popt [1]
c = popt [2]
A_0_vaccin=pop [0]
k1=pop [1]
c_vaccin=pop [2]


In [ ]:
etat={0:"M",
       1:"S",
       2:"V",
       3:"I",
       5:"PV"}
# M :le premier etat cad protégé maternellement
# S :suceptible
# V : vacciné
# I : infecté
# PV : protegé par vaccination

# transitions possible M à S ;S à V; V à PV; V à I; S à I

In [ ]:
def A_vaccin0(t):
  A_vaccin0 =float(nombre_anticorps) * np.exp(-k*t)+c
  return A_vaccin0

def A_t_vaccin(x,t):
  A_t_vaccin= A_vaccin0(x) * np.log(k1 * (t-x) )+ c_vaccin
  return A_t_vaccin

def temps_pour_degradation(A_0, k , c ,m ):
    A_0 = float(A_0)
    t = -(1/k ) * np.log((m - c ) / A_0)
    return t

def reponse(x ,k1,c_vaccin,m):
  t =  (1/k1) * np.exp((m - c_vaccin) / A_vaccin0(x))+x
  return t

#intervalle de transition de l'etat elevé à l'état moyen
nombre_anticorps = input("Entrez le nombre d'anticorps pour le jour 1 : ")
seuil_M= int(nombre_anticorps)* 0.2
cut_offFLUNPS=668
#intervalle de temps de l'etat M _on considére que le poussin va rester dans l'etat M es sa naissance jusqu'à perdre 80% des titres des MOD du premier jour
intervalle_M = (1,round(temps_pour_degradation(nombre_anticorps, k , c ,seuil_M)))
print('intervalle de temps pour passer de etat M à etat S: ', intervalle_M)
#sejour de M avant de passer à S
S_MS=intervalle_M[1]-intervalle_M[0]

#pour aller de l'etat suceptible à l'etat infecté on doit avoir le nombre des anticorps qui inferieur du cutt_of FLUNPS ;donc l'intervalle de temps de S avant de passer à I va etre calculer comme suit
intervalle_SI=(round(temps_pour_degradation(nombre_anticorps, k , c ,seuil_M))+1 ,round(temps_pour_degradation(nombre_anticorps, k , c ,cut_offFLUNPS)))
print('intervalle de temps pour passer de etat S à etat I: ', intervalle_SI)
#sejour de S avant de passer à I
S_SI=intervalle_SI[1]-intervalle_SI[0]

#on sait que l'age de vaccination doit etre dans l'intevalle [14;21] donc le temps qu'on va rester dans l'etat S va depondre de notre choix de l'age de vaccination
age_vaccin= [14,15,16,17,18,19,20,21]
sejour_SV=[]
sejour_VPV=[]
sejour_VI=[]

for age in age_vaccin:
    intervalle_SV = (round(temps_pour_degradation(nombre_anticorps, k, c, seuil_M)) + 1, age-1)
    print(f"Intervalle pour passer de etat S à V pour age de vaccination {age}: {intervalle_SV}")
    #sejour de S avant de passer à V
    S_SV=intervalle_SV[1]-intervalle_SV[0]
    sejour_SV.append(S_SV)

    #pour determiner l'intervalle de temps nesséssaire pour passer de etat V à VP on va mettre l'hyphotese que le poussin arrive à etre protege si il a reussi non seulement à developper une rponse immunitaire mais aussi d'avoir un nombre des anticorps qui est superieur au cutt_off
    intervalle_VPV=(age ,round(reponse(age+14,k1,c_vaccin,cut_offFLUNPS)))
    print(f"Intervalle pour passer de etat V à PV pour age de vaccination {age}: {intervalle_VPV}")
    #sejour de V avant de passer à PV
    S_VPV=intervalle_VPV[1]-intervalle_VPV[0]
    sejour_VPV.append(S_VPV)

    #pour determiner l'intervalle de temps pour passer de etat V à I ;si le poussin au jour de la vaccination a un nombre des anticorps qui est pas inferieur au cutt_off on va considerer que l'intervalle va etre du jour de vaccination jusqu'au le jour ou on a le cutt_off ;et si le poussin a eu un nombre inferieur des anticorps donc le poussin peut etre infecte meme apres la vaccination par 1j on peut prendre 2 ou 3j ,car dans cette periode il n'a pas encore arriver à developper une reponse immunitaire
    if A_vaccin0(age)> cut_offFLUNPS :
      intervalle_VI = (age,round(temps_pour_degradation(nombre_anticorps, k , c ,cut_offFLUNPS )))
      print(f"Intervalle pour passer de etat V à I pour age de vaccination {age}: {intervalle_VI}")
      #sejour de V avant de passer à I
      S_VI=intervalle_VI[1]-intervalle_VI[0]
      sejour_VI.append(S_VI)
    else :
      intervalle_VI=(age,age+2)
      print(f"Intervalle pour passer de etat V à I pour age de vaccination {age}: {intervalle_VI}")
      #sejour de V avant de passer à I
      S_VI=intervalle_VI[1]-intervalle_VI[0]
      sejour_VI.append(S_VI)




Entrez le nombre d'anticorps pour le jour 1 : 15000
intervalle de temps pour passer de etat M à etat S:  (1, 6)
intervalle de temps pour passer de etat S à etat I:  (7, 19)
Intervalle pour passer de etat S à V pour age de vaccination 14: (7, 13)
Intervalle pour passer de etat V à PV pour age de vaccination 14: (14, 29)
Intervalle pour passer de etat V à I pour age de vaccination 14: (14, 19)
Intervalle pour passer de etat S à V pour age de vaccination 15: (7, 14)
Intervalle pour passer de etat V à PV pour age de vaccination 15: (15, 30)
Intervalle pour passer de etat V à I pour age de vaccination 15: (15, 19)
Intervalle pour passer de etat S à V pour age de vaccination 16: (7, 15)
Intervalle pour passer de etat V à PV pour age de vaccination 16: (16, 31)
Intervalle pour passer de etat V à I pour age de vaccination 16: (16, 19)
Intervalle pour passer de etat S à V pour age de vaccination 17: (7, 16)
Intervalle pour passer de etat V à PV pour age de vaccination 17: (17, 32)
Intervalle po

In [ ]:
print(sejour_SV)
print(sejour_VPV)
print(sejour_VI)

[6, 7, 8, 9, 10, 11, 12, 13]
[15, 15, 15, 15, 15, 15, 15, 15]
[5, 4, 3, 2, 1, 2, 2, 2]


In [ ]:
#etats absorbants
def etats_absorabnts(matrix):
	l=len(matrix)
	absorbingStates=[]
	for i in range(l):
		if matrix[i][i]==1:
			absorbingStates.append(i)

	return absorbingStates

In [ ]:
#etats transitoire
def etats_transition(matrix):
	l=len(matrix)
	transientStates=[]
	for i in range(l):
		if matrix[i][i]!=1:
			transientStates.append(i)

	return transientStates

In [ ]:
# Q matrix
def Q_matrice(trs,asbo):
    QMatrice=[]
    for i in range(len(trs)):
        totalrow=matrice_transition[trs[i]]
        # print(totalrow)
        Rtemp=[]
        for j in range(len(trs)):
            Rtemp.append(totalrow[trs[j]])
        QMatrice.append(Rtemp)
    return QMatrice

In [ ]:
#matrice fondamentale
def matrice_fondamentale(matrice):
  Qmatrix=Q_matrice(etats_transition(matrice ),etats_absorabnts(matrice ))

  identity=I = np.identity (len(Qmatrix))
  L= identity - Qmatrix
  N=np.linalg.inv(L)
  return N

In [ ]:
#matrice R
def R_matrice(trs,abso):
    RMatrice=[]
    for i in range(len(trs)):
        totalrow=matrice_transition[trs[i]]
        # print(totalrow)
        Rtemp=[]
        for j in range(len(abso)):
            Rtemp.append(totalrow[abso[j]])
        RMatrice.append(Rtemp)
    return RMatrice

In [ ]:
# probabilité d'absorption:
# B = (I - Q)^-1 * R

def matrice_probabilité_absorption(N,R):
  B= np.dot(N,R)
  return B

In [ ]:
#matrice generatrice
for i in range(0,len(age_vaccin)-1):
  print("pour age de vaccination: ", 14+i)
  # Construction de la matrice de transition
  print("matrice generatrice")
  #taux d'infection sans vaccination 0.5
  #taux d'echec de vaccination 0.02
  ####probleme///
  #sejour_SV : sejour dans S avant de passer à l'etat V (commence dès le jour qu'il devient suceptible jusqu'au jour de vaccination)
  #sejour_SI : sejour dans S avant de passer à l'etat I ((commence dès le jour qu'il devient suceptible jusqu'au jour de Infection sachant que si on a un no,bre des anticorps inferieur au cutt_off on a une grande probabilité d'etre infecté )
  #sejour_VI:sejour dans V avant de passer à l'etat I(commence des le jour de vaccination)cad que le poussin a ete infecter avant de developper une reponse immunitaire
  #sejour_VPV:sejour dans V avant de passer à l'etat PV ((commence des le jour de vaccination jusqu'à arriver à developper une reponse immunitaire suffisante cad superieur au cutt-off)
  matrice_generatrice = np.array([[ -1/S_MS , 1/S_MS , 0,0, 0], #M
                               [0 , -1/(sejour_SV[i]+S_SI) ,1/(sejour_SV[i]+S_SI)*0.5 , 1/(sejour_SV[i]+S_SI)*0.5 , 0],#S
                               [0, 0, -1/(sejour_VI[i]+sejour_VPV[i]) , 1/(sejour_VI[i]+sejour_VPV[i])*0.02 ,1/(sejour_VI[i]+sejour_VPV[i])*0.98 ],#V
                               [0 , 0, 0 , 0, 0 ],#I
                               [0, 0 ,0 ,0 ,0]#PV
                               ])
  print(matrice_generatrice)

  print('matrice de transition')
  # 0 si i=j ; si i!=j :  a(i,j)/-lambda_i ; et si i=j avec i absorbant donc c'est 1
  #dans notre cas on a 2 etat absorants sont I et PV
  matrice_transition =np.array([[0 , 1  , 0,0, 0],
                               [0 , 0 , 0.5 , 0.5 , 0],
                               [0, 0, 0 , 0.02 , 0.98 ],
                               [0 , 0, 0 , 1, 0 ],
                               [0, 0 ,0 ,0 ,1]
                               ])
  print(matrice_transition)

  #etats absorbants
  etats_absorabnts(matrice_transition)

  #etats transitoires
  etats_transition (matrice_transition)

  #matrice Q
  Qmatrix=Q_matrice(etats_transition(matrice_transition),etats_absorabnts(matrice_transition))

  #matrice identité
  identity=I = np.identity (len(Qmatrix))

  #matrice fondamentale
  matrice_fondamentale(matrice_transition)

  #matrice R
  R_matrice(etats_transition(matrice_transition),etats_absorabnts(matrice_transition))

  #matrice des probabilité d'absorption
  print("matrice de probabilités d absorption")
  matrice_probabilité=matrice_probabilité_absorption(matrice_fondamentale(matrice_transition),R_matrice(etats_transition(matrice_transition),etats_absorabnts(matrice_transition)))
  print(matrice_probabilité)



pour age de vaccination:  14
matrice generatrice
[[-0.2         0.2         0.          0.          0.        ]
 [ 0.         -0.05555556  0.02777778  0.02777778  0.        ]
 [ 0.          0.         -0.05        0.001       0.049     ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
matrice de transition
[[0.   1.   0.   0.   0.  ]
 [0.   0.   0.5  0.5  0.  ]
 [0.   0.   0.   0.02 0.98]
 [0.   0.   0.   1.   0.  ]
 [0.   0.   0.   0.   1.  ]]
matrice de probabilités d absorption
[[0.51 0.49]
 [0.51 0.49]
 [0.02 0.98]]
pour age de vaccination:  15
matrice generatrice
[[-0.2         0.2         0.          0.          0.        ]
 [ 0.         -0.05263158  0.02631579  0.02631579  0.        ]
 [ 0.          0.         -0.05263158  0.00105263  0.05157895]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
matrice de transition
[[0.   1

In [ ]:
#matrice generatrice
for i in range(0,len(age_vaccin)-1):
  print("pour age de vaccination: ", 14+i)
  # Construction de la matrice de transition
  print("matrice generatrice")
  #taux d'infection sans vaccination 0.5
  #taux d'echec de vaccination 0.02
  matrice_generatrice = np.array([[ S_MS , S_MS , 0,0, 0],
                               [0 , -sejour_SV[i]-S_SI , sejour_SV[i] ,S_SI , 0],
                               [0, 0, -sejour_VI[i] -sejour_VPV[i] , sejour_VI[i] ,sejour_VPV[i] ],
                               [0 , 0, 0 , 0, 0 ],
                               [0, 0 ,0 ,0 ,0]
                               ])
  print(matrice_generatrice)

  print('matrice de transition')
  # 0 si i=j ; si i!=j :  a(i,j)/-landa_i ; et si i=j avec i absorbant donc c'est 1
  #dans notre cas on a 2 etat absorants sont I et PV
  matrice_transition =np.array([[0 , S_MS/S_MS  , 0,0, 0],
                               [0 , 0 , sejour_SV[i]/(sejour_SV[i]+S_SI) , S_SI/(sejour_SV[i]+S_SI) , 0],
                               [0, 0, 0 , sejour_VI[i]/(sejour_VI[i]+sejour_VPV[i]) ,sejour_VPV[i]/(sejour_VI[i] +sejour_VPV[i]) ],
                               [0 , 0, 0 , 1, 0 ],
                               [0, 0 ,0 ,0 ,1]
                               ])
  print(matrice_transition)

  #etats absorbants
  etats_absorabnts(matrice_transition)

  #etats transitoires
  etats_transition (matrice_transition)

  #matrice Q
  Qmatrix=Q_matrice(etats_transition(matrice_transition),etats_absorabnts(matrice_transition))

  #matrice identité
  identity=I = np.identity (len(Qmatrix))

  #matrice fondamentale
  matrice_fondamentale(matrice_transition)

  #matrice R
  R_matrice(etats_transition(matrice_transition),etats_absorabnts(matrice_transition))

  #matrice des probabilité d'absorption
  print("matrice de probabilités d absorption")
  matrice_probabilité=matrice_probabilité_absorption(matrice_fondamentale(matrice_transition),R_matrice(etats_transition(matrice_transition),etats_absorabnts(matrice_transition)))
  print(matrice_probabilité)



pour age de vaccination:  14
matrice generatrice
[[  5   5   0   0   0]
 [  0 -18   6  12   0]
 [  0   0 -20   5  15]
 [  0   0   0   0   0]
 [  0   0   0   0   0]]
matrice de transition
[[0.         1.         0.         0.         0.        ]
 [0.         0.         0.33333333 0.66666667 0.        ]
 [0.         0.         0.         0.25       0.75      ]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         0.         0.         1.        ]]
matrice de probabilités d absorption
[[0.75 0.25]
 [0.75 0.25]
 [0.25 0.75]]
pour age de vaccination:  15
matrice generatrice
[[  5   5   0   0   0]
 [  0 -19   7  12   0]
 [  0   0 -19   4  15]
 [  0   0   0   0   0]
 [  0   0   0   0   0]]
matrice de transition
[[0.         1.         0.         0.         0.        ]
 [0.         0.         0.36842105 0.63157895 0.        ]
 [0.         0.         0.         0.21052632 0.78947368]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         0.